<a href="https://colab.research.google.com/github/kirbu123/FilmGeneration/blob/develop/FilmGenTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libs

In [8]:
import torch
import time
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, KFold
from sklearn.multiclass import OneVsRestClassifier
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import csv
from urllib.request import urlopen
import multiprocessing

## Cloning git repository

In [ ]:
!git clone https://github.com/huggingface/diffusers
!cd diffusers
!pip install -e .

Cloning into 'diffusers'...
remote: Enumerating objects: 38856, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 38856 (delta 119), reused 152 (delta 88), pack-reused 38633
Receiving objects: 100% (38856/38856), 25.78 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (28701/28701), done.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


## Some setting

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

## Download test dataset

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/dog'

## Making target directory

In [ ]:
!cd /content
!mkdir -p model
!pwd

/content


In [12]:
!pip install diffusers

  Using cached diffusers-0.21.3-py3-none-any.whl (1.5 MB)


## !!!CHANGE FIRST ROWS OF /content/diffusers/examples/dreambooth/train_dreambooth.py .PY FILE!!!

1) Change 55 row to:


In [22]:
import diffusers.training_utils

2) Cahge 65 row to:

In [ ]:
# check_min_version("0.22.0.dev0")

## Finetune model

In [21]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export INSTANCE_DIR="/content/dog"
!export OUTPUT_DIR="/content/model"

!accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4"  \
  --instance_data_dir="/content/dog" \
  --output_dir="/content/model" \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400

2023-09-27 21:25:13.938956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
09/27/2023 21:25:16 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'dynamic_thresholding_ratio', 'thresholding', 'prediction_type', 'sample_max_value', 'variance_type', 'timestep_spacing', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
{'norm_num_groups', 'force_upcast'} was not found in config. Values will be initialized to default values.
{'cross_attention_norm', 'transformer_layers_per_block', 'dropout', 'conv_in_kernel', 'dual_cross_attention', 'mid_block_only_cross_attention', 'addition_embed_type_num_heads', 'time_embedding_type', 'addition_time_embed_dim', 'class

## Show results

In [ ]:
from diffusers import *
import torch

model_id = "path-to-your-trained-model"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("dog-bucket.png")